# Import

In [1]:
import sys
%matplotlib inline
SCRIPTS_PATH = '../scripts'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import regression_model
import NNModel
from tagging import *
from NNModel import *
from window_aggregation import *
from dull_model import *
from collocation import *

Using TensorFlow backend.


# Load test data

In [11]:
path_to_data = '../data/proba_1_v5.csv'
test_df = pd.read_csv(path_to_data).fillna(' ')

In [9]:
test_df.head()

,Unnamed: 0,PRODUCT,NAME,CATEGORY_ID,BRAND_ID,CATEGORY_NAME,BRAND_NAME,TYPE,SUBJECT,TEXT,POSTED_DATE,RATING,BENEFITS,DRAWBACKS,RECOMMENDED,LIKES_COUNT,DISLIKES_COUNT
0,0,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,"Небольшая симпатичная морозильная камера, нашл...",02.09.2013,4.0,NaN,NaN,0,30,26
1,1,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,Пользуемся месяц. Искали недорогую маленькую м...,17.09.2017,5.0,низкая цена по сравнению с другими с такими же...,Пока не нашла,0,1,0
2,2,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,"Морозилка хорошая, подошла по высоте кухонного...",25.08.2017,4.7,NaN,NaN,0,1,0
3,3,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,4 года назад началась моя любовь с Либхером с ...,07.08.2017,5.0,Морозит))),"За 4 года выросла цена в 1,5 раза",0,1,1
4,4,20025017,Морозильная камера Liebherr G 1213-20 ...,4020301,313,FREEZERS,Liebherr,review,NaN,Искал родителям для использования на даче. Уда...,26.02.2017,4.7,"Компактный морозильник. Вместительные ящики, к...",Пока не выявлены,0,1,0


# Init models

In [3]:
nn_model = nn_init('../model/')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
tag_model = tag_init('../model/dn/')

In [5]:
dull_model = DullModel('../model/rm/')

# Predict

In [12]:
products = test_df['PRODUCT'].unique()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
for product in products:
    comments = test_df[test_df['PRODUCT'] == product]['TEXT'].tolist()
    prod_info = test_df[test_df['PRODUCT'] == product].head(1)
    prod_name = prod_info['NAME'].tolist()[0].strip() if len(prod_info['NAME']) > 0 else ''
    print('Продукт %d, %s' % (product, prod_name))
    
    # DN - tags
    num_chosen_tags = 3
    single_prod_df = test_df[test_df['PRODUCT'] == product]
    tags = tag_model.predict(single_prod_df, top_n=num_chosen_tags+1)
    tags = [t for t in tags if len(t.split(' ')) <=3]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower().strip() if c in alphabet])
        tag_flt = tag_flt[1:len(tag_flt)] if tag_flt[0] == ' ' else tag_flt
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    
    # coallocation tags
    num_chosen_coallocations = 3
    col_model = Collocation_find('../model/mk/stopw-1.csv', path_to_data)
    tags = col_model.predict(product)[:num_chosen_coallocations]
    tag_line = ''
    for tag in tags:
        tag_flt = ''.join([c for c in tag.lower().strip() if c in alphabet])
        tag_flt = tag_flt[1:len(tag_flt)] if tag_flt[0] == ' ' else tag_flt
        tag_line += '#%s ' % tag_flt.replace(' ', '_')
    print('\t' + tag_line)
    print(' ')
    
    # nn 
    wnd_agg = aggregate(comments, num_comments=3, model=nn_model, wnd_l=12, stride=6, positive_q=(0.8, 0.95), negative_threshold=3)
    print('\tПокупатели отмечают такие положительные моменты: ')
    if len(wnd_agg['+']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['+']:
            print('\t\t...%s...' % com)
    print(' ')
            
    print('\tПокупатели отмечают такие отрицательные моменты: ')
    if len(wnd_agg['-']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['-']:
            print('\t\t...%s...' % com)
    print(' ')
    
    # comments
    num_chosen_comments = 3
    dull_pres = dull_model.predict(single_prod_df, list(range(len(single_prod_df))))[:num_chosen_comments]
    print('\tПокупатели также находят важным: ')
    if len(dull_pres) == 0:
        print('\t\t---')
    else:
        for com in dull_pres:
            print('\t\t...%s...' % com)
    
    print(' ')        
    print(''.join(['*'] * 110))
    print(' ')

Продукт 20025017, Морозильная камера Liebherr G 1213-20
	#морозильник_не_шумный #морозилка_хорошая 
	#габаритам_прочными_выдвижными #прочными_выдвижными_ящиками #небольшая_габаритам_прочными 
 
	Покупатели отмечают такие положительные моменты: 
		...бесшумно вместительна на табло видна заданная температура а не которая в морозилке ...
		...собран качественно,каких то косяков в сборке и работе пока не наблюдаю,управление электронное, ...
		...отличная камера,рекомендую! ...
 
	Покупатели отмечают такие отрицательные моменты: 
		---
 
	Покупатели также находят важным: 
		...по  закону  прав  потребителей  нельзя  не  верьте  продавцам  если  только  само  руководство  магазина...
		...в  шоке  потом  подумал  про  транспортировочные  болты  на  следующий  день  наш  л  их...
		...меня  деревянные  полы  дсп  на  лагах  линолеум  если  у  вас  бетонные  полы  то...
 
**************************************************************************************************************
 
Продукт 30

	#мелкий_мусор 
	#пылесос_очень_понравился #аккумулятора_хватает_час #валики_щётке_вынимаются 
 
	Покупатели отмечают такие положительные моменты: 
		...мне пылесос очень понравился! три режима работы, легко чистится! даже валики на ...
		...это моя любовь с первого дня, у меня 3 этажа и общая ...
		...всем советую ...
 
	Покупатели отмечают такие отрицательные моменты: 
		---
 
	Покупатели также находят важным: 
		...говорят  привозить  на  диагностику  на  рабочих  дней  может  меньше  вместо  пылесоса  получили  проблему...
		...сломалось  мгновенно  в  сервисном  центре  консультаций  не  дают  а  сразу  говорят  привозить  на...
		...не  менее  часов  заряжался  ч  где  то  индикатор  заряда  показал  полную  зарядку  уже...
 
**************************************************************************************************************
 
Продукт 20030060, Пылесос с пылесборником Bosch GL-30 BGL32003
	#индикатор_заполнения 
	#ранее_писала_отзывы #руководство_пользователя_предлагает

	#стирает_очень_тихо #минутная_стирка 
	#прослужила_лет_решил #остановить_свой_выбор #свой_выбор_именно 
 
	Покупатели отмечают такие положительные моменты: 
		...акция была,и она полностью оправдала мои надежды!)раньше тоже стиралка самсунг была,но старенькая ...
		...надежды!)раньше тоже стиралка самсунг была,но старенькая и вещи просто застирывала уже,цвет у ...
		...так что я очень её рекомендую!! ...
 
	Покупатели отмечают такие отрицательные моменты: 
		...вещей грязный какой-то был,так новая с первого раза отстирала мои полотенца до ...
		...их изначального цвета,поставила программу хлопок + интенсивное замачивание,потом постирала белые вещи которые ...
		...интенсивное замачивание,потом постирала белые вещи которые уже были серо-белые и она выдала ...
 
	Покупатели также находят важным: 
		...обязательно  нужно  по  строительному  уровню  установить  машину  т  к  при  работе  центрифуги  на...
		...устанавливать  советую  на  антивибрационные  подставки...
		...достаточно  над